In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import copy
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sys.path.append('/users/mtaranov/LongRange3D/')
from models import RandomForest, SVC
from metrics import ClassificationResult
from utils import concat_motifs, get_features, get_labels, subsample_data, normalize_features, reconstruct_2d, printMatrix, binarize, zscore, get_2D, plot_prediction, change_scale 

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX TITAN X (CNMeM is enabled with initial size: 2500 MB, cuDNN 5005)


In [37]:
contacts='PE'
thres='5'
mean_or_max='max'

path_to_features='/users/mtaranov/get_peaks/peaks_at_window/np/'+contacts+'_thres'+thres+'_'+mean_or_max+'/'
path_to_labels='/users/mtaranov/datasets_3d/dist_matched_'+contacts+'/'

X_train = np.load(path+'X_train.npy')
X_test = np.load(path+'X_test.npy')
X_valid = np.load(path+'X_valid.npy')

y_train = get_labels(path_to_labels+'y_train_thres_10.npy')
y_test = get_labels(path_to_labels+'y_test_thres_10.npy')
y_valid = get_labels(path_to_labels+'y_valid_thres_10.npy')


In [38]:
def normalize_features(X_train, X_valid, X_test, normalizer=StandardScaler):
    # fit normalizer
    normalizer = normalizer().fit(X_train[:,:])
    # transform features
    X_train_new=copy.copy(X_train)
    X_valid_new=copy.copy(X_valid)
    X_test_new=copy.copy(X_test)
    X_train_new = normalizer.transform(X_train)
    X_valid_new = normalizer.transform(X_valid)
    X_test_new = normalizer.transform(X_test)

    return (X_train_new, X_valid_new, X_test_new)

In [39]:
X_train_normalized, X_valid_normalized, X_test_normalized = normalize_features(X_train, X_valid, X_test)

# Random Forest

In [40]:
X_train_normalized.shape

(4824, 10)

In [41]:
y_train.shape

(4824, 1)

In [42]:
rf = RandomForest()

In [43]:
rf.train(X_train_normalized, y_train)
preds_test = rf.predict(X_test_normalized)
preds_train = rf.predict(X_train_normalized)
print ('On Test:\n{}\n'.format(ClassificationResult(y_test, preds_test)))
print ('On Train:\n{}\n'.format(ClassificationResult(y_train, preds_train)))

On Test:
Balanced Accuracy: 51.10%	 auROC: 0.527	 auPRC: 0.533	 auPRG: 0.033
Recall at 5%|10%|20% FDR: 0.0%|0.0%|0.0%	 Num Positives: 906	 Num Negatives: 906	 

On Train:
Balanced Accuracy: 99.05%	 auROC: 0.999	 auPRC: 0.999	 auPRG: 0.999
Recall at 5%|10%|20% FDR: 99.9%|100.0%|100.0%	 Num Positives: 2412	 Num Negatives: 2412	 

